In [ ]:
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import pearsonr,zscore
import hickle as hkl
from sklearn.metrics import r2_score
from numpy.linalg import solve, svd, norm
from tqdm import tqdm

# Load Datasets

In [ ]:
cell_embedding = hkl.load("embeddings/final_X_tcga_processed.hkl")
cell_embedding /= norm(cell_embedding, axis=1).reshape(-1, 1)
gene_effects_df = hkl.load("datasets/2023/CRISPRGeneEffect_processed.hkl")

# Train Models

In [ ]:
def train_individual_rfm_cell():
    device = "cuda:3"
    bandwidth = 1
    reg = 1e-5

    X = torch.tensor(cell_embedding.values).to(device).float()

    num_cells = X.shape[0]
    knockouts = gene_effects_df.columns
    num_knockouts = len(knockouts)

    cell_distances = kernel.euclidean_distances(X,X).to(device)
    dist_ko = cell_distances.fill_diagonal_(0)
    y = torch.tensor(gene_effects_df.values).to(device).float()
    sol = torch.linalg.solve(torch.exp(-bandwidth*(dist_ko)**0.5).to(device) + reg*torch.eye(dist_ko.shape[0],device=device),y)

    return sol

In [ ]:
sol = train_individual_rfm_cell()

# Get Grads

In [ ]:
def euclidean_distances(samples, centers, M=None,squared=True,diag_only=False):
    '''Calculate the pointwise distance.
    Args:
        samples: of shape (n_sample, n_feature).
        centers: of shape (n_center, n_feature).
        squared: boolean.
    Returns:
        pointwise distances (n_sample, n_center).
    '''
    if M is None:
        samples_norm = torch.sum(samples**2, dim=1, keepdim=True)
    else:
        if diag_only:
            samples_norm = (samples * M) * samples
            # samples_cpu = samples.detach().cpu()
            # M_cpu = M.detach().cpu()
            # samples_norm = samples_cpu**2 * M_cpu
            # samples_norm = samples_norm.to("cuda:1")
        else:
            samples_norm = (samples @ M) * samples
        samples_norm = torch.sum(samples_norm, dim=1, keepdims=True)

    if samples is centers:
        centers_norm = samples_norm
    else:
        if M is None:
            centers_norm = torch.sum(centers**2, dim=1, keepdims=True)
        else:
            # centers_norm = (centers.diag() * M).diag() * centers
            if diag_only:
                centers_norm = (centers * M) * centers
            else:
                centers_norm = (centers @ M) * centers
            centers_norm = torch.sum(centers_norm, dim=1, keepdims=True)
    centers_norm = torch.reshape(centers_norm, (1, -1))

    distances = samples.mm(torch.t(centers))
    distances.mul_(-2)
    distances.add_(samples_norm)
    distances.add_(centers_norm)
    if not squared:
        distances.clamp_(min=0)
        distances.sqrt_()

    return distances

def laplace_kernel(samples, centers, bandwidth,M=None, diag_only=False):
    '''Laplacian kernel.
    Args:
        samples: of shape (n_sample, n_feature).
        centers: of shape (n_center, n_feature).
        bandwidth: kernel bandwidth.
    Returns:
        kernel matrix of shape (n_sample, n_center).
    '''
    assert bandwidth > 0
    kernel_mat = euclidean_distances(samples, centers, M=M, squared=False, diag_only=diag_only)
    kernel_mat.clamp_(min=0)
    gamma = 1. / bandwidth
    kernel_mat.mul_(-gamma)
    kernel_mat.exp_()
    return kernel_mat

In [ ]:
def get_grads(X, sol, P, L=1,centering=False,diag_only=True):
    
    K = laplace_kernel(X, X, bandwidth=1, M=P, diag_only=diag_only)

    dist = euclidean_distances(X, X, M=P, squared=False, diag_only=diag_only)
    dist.clamp_(min=0)
    dist[dist < 1e-10] = 0

    with np.errstate(divide='ignore'):
        K = K/dist

    K[K == float("Inf")] = 0.
    n,d = X.shape
    num_kos,n = sol.shape

    grads = torch.zeros((d,num_kos)).to(X.device)
    for i in tqdm(range(num_kos)):
        weight = sol[i,:].reshape((-1,1))

        step2 = K @ (weight * X)
        step3 = (weight.T @ K).T * X
        G = (step2 - step3) * -1/L
        G = torch.sum(G**2,axis=0)
        grads[:,i] = G/n

    return grads

In [ ]:
device = "cuda:3"
X = torch.tensor(cell_embedding.values).to(device).float()
n,d = X.shape
P = torch.ones(d).double().to(device)
grads = get_grads(X,sol.T,L=1,P=P,centering=False,diag_only=True)

# Add PCC Weights

In [ ]:
def get_pcc():
    embedding = hkl.load("embeddings/final_X_tcga_raw_unnormalized.hkl")
    features = embedding.columns
    num_cell_features = len(features)
    knockouts = gene_effects_df.columns
    feature_importance_df = {}

    for i in tqdm(range(len(knockouts))):
        knockout = knockouts[i]
        y = gene_effects_df[knockout]

        M_i = np.ones((len(features),))
        for j in range(len(embedding.columns)):
            x = embedding.columns[j]
            if x.split("_")[-1] == "exp": #exp feature
                ind = np.abs(zscore(embedding[x].values)) < 3
                r = np.corrcoef([embedding[x].values[ind], y[ind]])[0,1]
            else: #mut feature
                r = np.corrcoef([embedding[x].values, y])[0,1]
            
            M_i[j] *= r

        feature_importance_df[knockout] = M_i.flatten()
    
    feature_importance_df = pd.DataFrame(feature_importance_df,columns=knockouts,index=features)
    hkl.dump(feature_importance_df,"datasets/pcc.hkl")

In [ ]:
pcc = get_pcc().fillna(0)
pcc = pcc.loc[grads.index]
mut = [x for x in pcc.index if x.split("_")[-1] != "exp"]
pcc.loc[mut] = -(pcc.loc[mut].clip(upper=0))

exp = [x for x in pcc.index if x.split("_")[-1] == "exp"]
pcc.loc[exp] = abs(pcc.loc[exp])

feature_importance_df = grads * pcc
hkl.dump(feature_importance_df,"datasets/feature_importances.hkl")